# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline

### 1. Import libraries and load data from database.
* Import Python libraries
* Load dataset from database with read_sql_table
* Define feature and target variables X and Y

In [43]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [44]:
# Import libraries 
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', 500)

import sys
import os
import re
import string
from sqlalchemy import create_engine
import pickle

from scipy.stats import gmean

# import relevant functions/modules from the sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

In [45]:
# Load data from database
database_filepath = '../data/disaster_response_db.db'
engine = create_engine('sqlite:///' + database_filepath)
table_name = 'T_disaster_df'
df = pd.read_sql_table(table_name, engine)

In [46]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.0,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000,26215.000000
mean,15224.871333,0.766470,0.170666,0.004501,0.414267,0.079496,0.050086,0.027618,0.017967,0.032806,0.0,0.063780,0.111501,0.088270,0.015449,0.023040,0.011368,0.033378,0.045546,0.131451,0.065039,0.045813,0.050849,0.020294,0.006065,0.010795,0.004578,0.011787,0.043906,0.278352,0.082205,0.093191,0.010757,0.093649,0.020217,0.052489,0.193591
std,8827.053788,0.423085,0.376224,0.066941,0.492604,0.270517,0.218126,0.163878,0.132833,0.178131,0.0,0.244366,0.314758,0.283693,0.123333,0.150034,0.106013,0.179625,0.208503,0.337900,0.246600,0.209084,0.219693,0.141006,0.077645,0.103340,0.067504,0.107929,0.204890,0.448196,0.274682,0.290705,0.103160,0.291345,0.140746,0.223015,0.395120
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15663.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.500000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
# Remove child alone, column of 0's
df = df.drop(['child_alone'], axis = 1)

In [13]:
df.groupby("related").count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,20093,20093,6775,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093,20093


In [47]:
# Extract X and y variables from the data for modelling
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data


In [48]:
def tokenize(text):
    """
    tokenize intents to preprocess the text features, tokenizes and
    lemmatizes each message.

    :text: Dataframe of text features
    
    :return:  clean_tokens as a DataFrame that contains each message
              preprocessed for modelling
    """
    # Convert to lowercase
    text_lw = text.lower()

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text_lw)

    # Lemmanitizer to get the 'grammatical root'  of each word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens standarized
    tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    
    # Remove stop words
    nltk_stop_words = nltk.corpus.stopwords.words('english')
    tokens_ns = [t for t in tokens if t not in nltk_stop_words]
    
    # Remove punctuation
    clean_tokens = [w for w in tokens_ns if w not in string.punctuation]
   
    return clean_tokens

### Debug functions to understand effects

In [69]:
# tokenize(X[1])
text = X[7]
print(text)
text_lw = text.lower()

# Extract the word tokens from the provided text
tokens = nltk.word_tokenize(text_lw)
print(tokens)
# Lemmanitizer to get the 'grammatical root'  of each word
lemmatizer = nltk.WordNetLemmatizer()

# List of clean tokens standarized
tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
print(tokens)
# Remove stop words
nltk_stop_words = nltk.corpus.stopwords.words('english')
tokens_ns = [t for t in tokens if t not in nltk_stop_words]

# Remove punctuation
clean_tokens = [t for t in tokens_ns if t not in string.punctuation]
print(clean_tokens)


Please, we need tents and water. We are in Silo, Thank you!
['please', ',', 'we', 'need', 'tents', 'and', 'water', '.', 'we', 'are', 'in', 'silo', ',', 'thank', 'you', '!']
['please', ',', 'we', 'need', 'tent', 'and', 'water', '.', 'we', 'are', 'in', 'silo', ',', 'thank', 'you', '!']
['please', 'need', 'tent', 'water', 'silo', 'thank']


### 3. Build a machine learning pipeline
This machine pipeline should take in the message column as input and output classification results on the other 36 categories in the dataset. You may finde the MultiOutputClassifier helpful for predicting multiple target variables.

In [49]:
pipeline = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [
                    (
                        "text_pipeline",
                        Pipeline(
                            [
                                (
                                    "count_vectorizer",
                                    CountVectorizer(tokenizer=tokenize),
                                ),
                                ("tfidf_transformer", TfidfTransformer()),
                            ]
                        ),
                    ),
                ]
            ),
        ),
        ("classifier", MultiOutputClassifier(RandomForestClassifier())),
    ]
)

### 4. Train pipeline
* Split data into train and test sets
* Train pipeline

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline.fit(X_train, y_train)

### 5. Test your model

report the F1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's classification_report on each.

In [51]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

In [52]:
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4982
               request       0.82      0.48      0.60      1137
                 offer       0.00      0.00      0.00        30
           aid_related       0.76      0.69      0.72      2669
          medical_help       0.66      0.06      0.11       528
      medical_products       0.77      0.08      0.15       335
     search_and_rescue       0.59      0.07      0.13       182
              security       1.00      0.01      0.02       119
              military       0.73      0.04      0.07       212
           child_alone       0.00      0.00      0.00         0
                 water       0.87      0.38      0.53       405
                  food       0.84      0.62      0.71       735
               shelter       0.83      0.30      0.45       595
              clothing       0.77      0.10      0.18        98
                 money       1.00      

D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-sc

In [53]:
print(classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00     15111
               request       1.00      0.99      1.00      3337
                 offer       1.00      0.98      0.99        88
           aid_related       1.00      1.00      1.00      8191
          medical_help       1.00      0.99      1.00      1556
      medical_products       1.00      0.99      1.00       978
     search_and_rescue       1.00      1.00      1.00       542
              security       1.00      0.99      0.99       352
              military       1.00      1.00      1.00       648
           child_alone       0.00      0.00      0.00         0
                 water       1.00      1.00      1.00      1267
                  food       1.00      1.00      1.00      2188
               shelter       1.00      1.00      1.00      1719
              clothing       1.00      1.00      1.00       307
                 money       1.00      

### 6. Improve your model
Use grid search to find better parameters

In [56]:
# pipeline1.get_params().keys()
parameters_grid = {'classifier__estimator__max_depth': [10, 20, 30, 40, 50, 60],
              'classifier__estimator__n_estimators': [10, 20, 40]}

#parameters_grid = {'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x00000260870788B0>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__max_depth': [10, 20, 30, 40,
                                                              50, 60],
                         'classifie

Find the best parameters according to grid

In [58]:
cv.best_params_

{'classifier__estimator__max_depth': 60,
 'classifier__estimator__n_estimators': 10}

In [60]:
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

In [61]:
print(classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      1.00      0.89     15111
               request       0.99      0.45      0.62      3337
                 offer       1.00      0.65      0.79        88
           aid_related       0.94      0.68      0.79      8191
          medical_help       1.00      0.26      0.41      1556
      medical_products       1.00      0.31      0.48       978
     search_and_rescue       1.00      0.13      0.23       542
              security       1.00      0.11      0.20       352
              military       1.00      0.34      0.51       648
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.44      0.61      1267
                  food       0.99      0.45      0.62      2188
               shelter       1.00      0.36      0.52      1719
              clothing       1.00      0.44      0.61       307
                 money       1.00      

D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Python\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-sc

# 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [62]:
# Print the classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.78      0.99      0.87      4982
               request       0.88      0.28      0.43      1137
                 offer       0.00      0.00      0.00        30
           aid_related       0.76      0.55      0.64      2669
          medical_help       0.70      0.04      0.07       528
      medical_products       0.87      0.04      0.07       335
     search_and_rescue       1.00      0.02      0.03       182
              security       0.00      0.00      0.00       119
              military       0.88      0.03      0.06       212
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.11      0.20       405
                  food       0.89      0.23      0.36       735
               shelter       0.72      0.10      0.17       595
              clothing       0.83      0.05      0.10        98
                 money       1.00      

### Save model as a pickle file

In [20]:
m = pickle.dumps('classifier.pkl')